# Test model
Handling Panns cnn14

In [20]:
from audiointerp.dataset.esc50 import ESC50dataset
from audiointerp.model.cnn14 import TransferCnn14
import torch.nn as nn
import torch.optim as optim
import torchaudio.transforms as T
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [21]:
root_dir = "/home/yuliya/ESC50"
sr = 32000
train_folds = [1, 2, 3]
valid_folds = [4]
test_folds = [5]

In [22]:
n_fft = 1024
hop_length = 320
win_length = 1024
n_mels = 64
f_min = 50
f_max = 14000

In [23]:
spec = T.Spectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length, power=2.0)
mel = T.MelScale(n_mels=n_mels, sample_rate=sr, f_min=f_min, f_max=f_max, n_stft=n_fft // 2 + 1)
amplitude_to_db = T.AmplitudeToDB(stype="power", top_db=80)

In [24]:
feature_extractor = nn.Sequential(spec, mel, amplitude_to_db)

In [25]:
train_data = ESC50dataset(root_dir=root_dir, sr=sr, folds=train_folds, feature_extractor=feature_extractor)
valid_data = ESC50dataset(root_dir=root_dir, sr=sr, folds=valid_folds, feature_extractor=feature_extractor)
test_data = ESC50dataset(root_dir=root_dir, sr=sr, folds=test_folds, feature_extractor=feature_extractor)

In [26]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [27]:
def train_step(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    
    for samples, labels in dataloader:
        samples = samples.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(samples)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * samples.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += samples.size(0)
    
    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples
    
    return epoch_loss, epoch_acc.item()

In [28]:
def valid_step(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    
    with torch.no_grad():
        for samples, labels in dataloader:
            samples = samples.to(device)
            labels = labels.to(device)
            
            outputs = model(samples)
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * samples.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_samples += samples.size(0)
    
    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples
    
    return epoch_loss, epoch_acc.item()

In [29]:
def train(model, num_epochs, criterion, optimizer, device):
    best_acc = 0.0
    best_model = None


    for epoch in tqdm(range(num_epochs), desc='Epoch'):
    
        train_loss, train_acc = train_step(model, criterion, optimizer, train_loader, device)
        val_loss, val_acc = valid_step(model, criterion, valid_loader, device)
    
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f},   Val Acc:   {val_acc:.4f}")
    
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = model.state_dict()

    print(f"Best val Acc: {best_acc:.4f}")

    if best_model is not None:
        model.load_state_dict(best_model)
    
    torch.save(model.state_dict(), "best.pth")
    print("Модель сохранена в best.pth")

    return model

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
num_epochs = 15
learning_rate = 1e-3

In [32]:
model = TransferCnn14(num_classes=50)
model.load_base_weights("weights/Cnn14_mAP=0.431.pth")

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
model = model.to(device)

In [35]:
model = train(model, num_epochs, criterion, optimizer, device)

Epoch:   7%|▋         | 1/15 [00:35<08:14, 35.29s/it]

Train Loss: 2.8587, Train Acc: 0.3308
Val   Loss: 0.8394,   Val Acc:   0.8125


Epoch:  13%|█▎        | 2/15 [01:11<07:46, 35.86s/it]

Train Loss: 0.6353, Train Acc: 0.8133
Val   Loss: 0.7928,   Val Acc:   0.8475


Epoch:  20%|██        | 3/15 [01:47<07:13, 36.12s/it]

...

Epoch:  93%|█████████▎| 14/15 [08:56<00:39, 39.47s/it]

Train Loss: 0.0892, Train Acc: 0.9742
Val   Loss: 0.5796,   Val Acc:   0.8750


Epoch: 100%|██████████| 15/15 [09:35<00:00, 38.36s/it]

Train Loss: 0.0617, Train Acc: 0.9825
Val   Loss: 0.5929,   Val Acc:   0.8875
Best val Acc: 0.9100


Модель сохранена в best.pth


In [17]:
model.load_state_dict(torch.load("best.pth"))

<All keys matched successfully>

In [19]:
test_loss, test_acc = valid_step(model, criterion, test_loader, device)
print(f"Test loss: {test_loss:.2f}, Test acc: {test_acc:.2f}")

Test loss: 0.93, Test acc: 0.84
